In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
RAW_DATA_PATH = "~/library/data/raw/"
PROC_DATA_PATH = '~/library/data/processed/'

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
df_1 = pd.read_csv(RAW_DATA_PATH + 'business-demographics.csv', sep=',') 
df_2 = pd.read_csv(RAW_DATA_PATH + 'Jobs and Job Density.csv', sep=',') 
df_3 = pd.read_csv(RAW_DATA_PATH + 'mb-unemployment-rates.csv', sep=',', na_values=[' ', '-'])

In [3]:
def replaceCol(df):
    col_length = len(df.columns)
    mapping = {df.columns[0]:'code', df.columns[1]:'area'}
    for index in range(2, col_length):
        if index%2!=0:
            mapping[df.columns[index]] = '+/- Difference'
    return mapping

df_3.dropna(inplace=True, axis=0, how="any")
mapping = replaceCol(df_3)
df_3.rename(columns=mapping, inplace=True)
df_3 = df_3.reset_index(drop=True)
df = pd.merge(df_1, df_2, on=['code','area', 'year'])

In [4]:
def colToDrop(df):
    col_length = len(df.columns)
    arr = []
    for index in range(2, col_length):
        if df.columns[index].find('Jan') != -1:
            if index + 2 < col_length:
                index += 2
            pass
        arr.append(df.columns[index])    
    return arr

df_3.drop(columns=colToDrop(df_3), inplace=True, axis=1)

['Apr 2004-Mar 2005 ', '+/- Difference', 'Apr 2004-Mar 2005 ', '+/- Difference', 'Jul 2004-Jun 2005 ', '+/- Difference', 'Oct 2004-Sep 2005 ', '+/- Difference', 'Apr 2005-Mar 2006 ', '+/- Difference', 'Apr 2005-Mar 2006 ', '+/- Difference', 'Jul 2005-Jun 2006 ', '+/- Difference', 'Oct 2005-Sep 2006 ', '+/- Difference', 'Apr 2006-Mar 2007 ', '+/- Difference', 'Apr 2006-Mar 2007 ', '+/- Difference', 'Jul 2006-Jun 2007 ', '+/- Difference', 'Oct 2006-Sep 2007 ', '+/- Difference', 'Apr 2007-Mar 2008', '+/- Difference', 'Apr 2007-Mar 2008', '+/- Difference', 'Jul 2007-Jun 2008', '+/- Difference', 'Oct 2007-Sep 2008', '+/- Difference', 'Apr 2008-Mar 2009', '+/- Difference', 'Apr 2008-Mar 2009', '+/- Difference', 'Jul 2008-Jun 2009', '+/- Difference', 'Oct 2008-Sep 2009', '+/- Difference', 'Apr 2009-Mar 2010', '+/- Difference', 'Apr 2009-Mar 2010', '+/- Difference', 'Jul 2009-Jun 2010', '+/- Difference', 'Oct 2009-Sep 2010', '+/- Difference', 'Apr 2010-Mar 2011', '+/- Difference', 'Apr 2010-Ma

In [8]:
df_3 = df_3.melt(id_vars=["code", "area"], 
        var_name="Date", 
        value_name="Value")

In [11]:
df_3['Date'] = df_3['Date'].apply(lambda x: [int(s) for s in str.split(x) if s.isdigit()][0])

In [16]:
df_3.rename(columns={'Date':'year', 'Value':'unemployment_rate'}, inplace=True)

In [18]:
df = pd.merge(df, df_3, on=['code','area', 'year'])

In [22]:
df = df.set_index(['year', 'area'])

In [23]:
df

code  active_enterprises  births  \
year area                                                            
2004 Barking and Dagenham    E09000002                3120     480   
     Barnet                  E09000003               19655    2435   
     Bexley                  E09000004                6450     940   
     Brent                   E09000005               11295    1800   
     Bromley                 E09000006               11655    1590   
     Camden                  E09000007               22910    2840   
     Croydon                 E09000008               11230    1630   
     Ealing                  E09000009               12830    2115   
     Enfield                 E09000010                9305    1400   
     Greenwich               E09000011                5605     975   
     Hackney                 E09000012                9285    1445   
     Hammersmith and Fulham  E09000013               10745    1865   
     Haringey                E09000014                8435    1345   
     Harrow                  E09000015                9965    1370   
     Havering                E09000016                7570    1055   
     Hillingdon              E09000017                9170    1215   
     Hounslow                E09000018                8645    1310   
     Islington               E09000019               12625    1870   
     Kensington and Chelsea  E09000020               11675    1660   
     Kingston upon Thames    E09000021                6680     880   
     Lambeth                 E09000022                9555    1850   
     Lewisham                E09000023                6230    1135   
     Merton                  E09000024                7765    1205   
     Newham                  E09000025                5160     965   
     Redbridge               E09000026                8115    1330   
     Richmond upon Thames    E09000027               10855    1475   
     Southwark               E09000028               10730    1945   
     Sutton                  E09000029                6485     855   
     Tower Hamlets           E09000030                9590    1855   
     Waltham Forest          E09000031                6805    1130   
...                                ...                 ...     ...   
2017 Havering                E09000016               11305    1535   
     Hillingdon              E09000017               15085    2105   
     Hounslow                E09000018               15195    2130   
     Islington               E09000019               22295    3705   
     Kensington and Chelsea  E09000020               15245    2090   
     Kingston upon Thames    E09000021                9825    1295   
     Lambeth                 E09000022               16940    2570   
     Lewisham                E09000023               11605    1850   
     Merton                  E09000024               13110    1915   
     Newham                  E09000025               14570    2875   
     Redbridge               E09000026               16635    2680   
     Richmond upon Thames    E09000027               15105    1885   
     Southwark               E09000028               19370    2770   
     Sutton                  E09000029                9570    1180   
     Tower Hamlets           E09000030               20545    3365   
     Waltham Forest          E09000031               12755    2095   
     Wandsworth              E09000032               20095    3040   
     Westminster             E09000033               59945    8260   
     North East              E12000001               75580    9545   
     North West              E12000002              297760   47465   
     East Midlands           E12000004              190090   22565   
     West Midlands           E12000005              230100   30735   
     East                    E12000006              295225   36935   
     London                  E12000007              607870   92300   
     South East              E12000008      

In [24]:
df.to_csv(PROC_DATA_PATH + 'borough-unemployment-and-business-data.csv')